### Clean and condense Portland 2024 CVRs for city council STV election.
### This notebook: District 2

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None)

import numpy as np

import maup
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
maup.progress.enabled = True

from tqdm import tqdm

In [2]:
D2_df = pd.read_csv("./Original CVRs/City_of_Portland__Councilor__District_2_2024_11_29_17_26_12.cvr.csv")

In [3]:
D2_df

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,"Choice_36_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Sameer Kanal:NON","Choice_36_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Sameer Kanal:NON","Choice_36_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Sameer Kanal:NON","Choice_36_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Sameer Kanal:NON","Choice_36_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Sameer Kanal:NON","Choice_36_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Sameer Kanal:NON","Choice_37_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Debbie Kitchin:NON","Choice_37_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Debbie Kitchin:NON","Choice_37_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Debbie Kitchin:NON","Choice_37_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Debbie Kitchin:NON","Choice_37_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Debbie Kitchin:NON","Choice_37_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Debbie Kitchin:NON","Choice_38_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Nat West:NON","Choice_38_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Nat West:NON","Choice_38_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Nat West:NON","Choice_38_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Nat West:NON","Choice_38_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Nat West:NON","Choice_38_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Nat West:NON","Choice_39_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Reuben Berlin:NON","Choice_39_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Reuben Berlin:NON","Choice_39_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Reuben Berlin:NON","Choice_39_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Reuben Berlin:NON","Choice_39_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Reuben Berlin:NON","Choice_39_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Reuben Berlin:NON","Choice_40_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Chris Olson:NON","Choice_40_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Chris Olson:NON","Choice_40_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Chris Olson:NON","Choice_40_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Chris Olson:NON","Choice_40_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Chris Olson:NON","Choice_40_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Chris Olson:NON","Choice_41_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Marnie Glickman:NON","Choice_41_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Marnie Glickman:NON","Choice_41_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Marnie Glickman:NON","Choice_41_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Marnie Glickman:NON","Choice_41_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Marnie Glickman:NON","Choice_41_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Marnie Glickman:NON","Choice_42_1:City of Portland, Councilor, District 2:1:Number of Winners 3:James Armstrong:NON","Choice_42_1:City of Portland, Councilor, District 2:2:Number of Winners 3:James Armstrong:NON","Choice_42_1:City of Portland, Councilor, District 2:3:Number of Winners 3:James Armstrong:NON","Choice_42_1:City of Portland, Councilor, District 2:4:Number of Winners 3:James Armstrong:NON","Choice_42_1:City of Portland, Councilor, District 2:5:Number of Winners 3:James Armstrong:NON","Choice_42_1:City of Portland, Councilor, District 2:6

### Note that this is all the voters in the city, not just in D2.

In [4]:
list(D2_df.columns)

['RowNumber',
 'BoxID',
 'BoxPosition',
 'BallotID',
 'PrecinctID',
 'BallotStyleID',
 'PrecinctStyleName',
 'ScanComputerName',
 'Status',
 'Remade',
 'Choice_36_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Sameer Kanal:NON',
 'Choice_36_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Sameer Kanal:NON',
 'Choice_36_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Sameer Kanal:NON',
 'Choice_36_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Sameer Kanal:NON',
 'Choice_36_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Sameer Kanal:NON',
 'Choice_36_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Sameer Kanal:NON',
 'Choice_37_1:City of Portland, Councilor, District 2:1:Number of Winners 3:Debbie Kitchin:NON',
 'Choice_37_1:City of Portland, Councilor, District 2:2:Number of Winners 3:Debbie Kitchin:NON',
 'Choice_37_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Debbie Ki

### Here are all the D2 candidates that the CVR contains rankings for:

In [5]:
D2_candidates = [
    'Sameer Kanal',
    'Dan Ryan',
    'Elana Pirtle-Guiney',
    'Tiffani Penson',
    'Michelle DePass',
    'Nat West',
    'Marnie Glickman',
    'Jonathan Tasini',
    'Bob Simril',
    'Mariah Hudson',
    'Michael (Mike) Marshall',
    'James Armstrong',
    'Chris Olson',
    'Debbie Kitchin',
    'Jennifer Park',
    'Nabil Zaghloul',
    'Will Mespelt',
    'Laura Streib',
    'Reuben Berlin',
    'Liz Taylor',
    'Sam Sachs',
    'Antonio Jamal PettyJohnBlue',
    'Uncertified Write In',
    'Write-in-123',
    'Write-in-124',
    'Write-in-125'
]

### Collect columns where 1st place, etc. votes are tallied so we can look for overvotes/undervotes while converting to a better format.

In [6]:
rank1_cols = [col for col in D2_df.columns if '1:Number' in col]
rank2_cols = [col for col in D2_df.columns if '2:Number' in col]
rank3_cols = [col for col in D2_df.columns if '3:Number' in col]
rank4_cols = [col for col in D2_df.columns if '4:Number' in col]
rank5_cols = [col for col in D2_df.columns if '5:Number' in col]
rank6_cols = [col for col in D2_df.columns if '6:Number' in col]

all_rank_cols = rank1_cols + rank2_cols + rank3_cols + rank4_cols + rank5_cols + rank6_cols

### This dataset has rows for ALL Portland voters; trim it down to voters who cast at least one vote.

In [7]:
D2_voters_df = D2_df[D2_df[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True)

### Add a column for each raw ranking:

In [8]:
D2_voters_df.insert(10, "Rank1 raw", None)
D2_voters_df.insert(11, "Rank2 raw", None)
D2_voters_df.insert(12, "Rank3 raw", None)
D2_voters_df.insert(13, "Rank4 raw", None)
D2_voters_df.insert(14, "Rank5 raw", None)
D2_voters_df.insert(15, "Rank6 raw", None)


### For each voter and each rank, record whether the voter overvoted or undervoted in that rank, or if they voted for exactly one candidate in that rank, record that candidate.

In [9]:
for i in tqdm(D2_voters_df.index):
    
    rank1_votes = D2_voters_df.loc[i, rank1_cols].sum()
    if rank1_votes == 0:
        D2_voters_df.loc[i, 'Rank1 raw'] = "skipped"
    elif rank1_votes > 1:
        D2_voters_df.loc[i, 'Rank1 raw'] = "overvote"
    elif rank1_votes == 1:
        this_vote_col = [col for col in rank1_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank1 raw'] = this_vote

    rank2_votes = D2_voters_df.loc[i, rank2_cols].sum()
    if rank2_votes == 0:
        D2_voters_df.loc[i, 'Rank2 raw'] = "skipped"
    elif rank2_votes > 1:
        D2_voters_df.loc[i, 'Rank2 raw'] = "overvote"
    elif rank2_votes == 1:
        this_vote_col = [col for col in rank2_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank2 raw'] = this_vote

    rank3_votes = D2_voters_df.loc[i, rank3_cols].sum()
    if rank3_votes == 0:
        D2_voters_df.loc[i, 'Rank3 raw'] = "skipped"
    elif rank3_votes > 1:
        D2_voters_df.loc[i, 'Rank3 raw'] = "overvote"
    elif rank3_votes == 1:
        this_vote_col = [col for col in rank3_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank3 raw'] = this_vote

    rank4_votes = D2_voters_df.loc[i, rank4_cols].sum()
    if rank4_votes == 0:
        D2_voters_df.loc[i, 'Rank4 raw'] = "skipped"
    elif rank4_votes > 1:
        D2_voters_df.loc[i, 'Rank4 raw'] = "overvote"
    elif rank4_votes == 1:
        this_vote_col = [col for col in rank4_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank4 raw'] = this_vote

    rank5_votes = D2_voters_df.loc[i, rank5_cols].sum()
    if rank5_votes == 0:
        D2_voters_df.loc[i, 'Rank5 raw'] = "skipped"
    elif rank5_votes > 1:
        D2_voters_df.loc[i, 'Rank5 raw'] = "overvote"
    elif rank5_votes == 1:
        this_vote_col = [col for col in rank5_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank5 raw'] = this_vote

    rank6_votes = D2_voters_df.loc[i, rank6_cols].sum()
    if rank6_votes == 0:
        D2_voters_df.loc[i, 'Rank6 raw'] = "skipped"
    elif rank6_votes > 1:
        D2_voters_df.loc[i, 'Rank6 raw'] = "overvote"
    elif rank6_votes == 1:
        this_vote_col = [col for col in rank6_cols if D2_voters_df.loc[i, col] == 1][0]
        this_vote = [name for name in D2_candidates if name in this_vote_col][0]
        D2_voters_df.loc[i, 'Rank6 raw'] = this_vote


100%|████████████████████████████████████| 77686/77686 [02:42<00:00, 478.29it/s]


### Next, clean the votes for VoteKit according to Portland's rules: overvotes and undervotes are both skipped and ballots are transferred to the next ranking.  If the same candidate receives multiple rankings, only keep the highest ranking.
### It appears that the categories 'Write-in-123', 'Write-in-124', and 'Write-in-125' are ALSO skipped and ballots are transferred to the next ranking.  (Equivalently, these "candidates" were eliminated first in a "Round 0" that occurred before the published tabulation rounds.)

In [10]:
D2_voters_df.insert(16, "Rank1 condensed", None)
D2_voters_df.insert(17, "Rank2 condensed", None)
D2_voters_df.insert(18, "Rank3 condensed", None)
D2_voters_df.insert(19, "Rank4 condensed", None)
D2_voters_df.insert(20, "Rank5 condensed", None)
D2_voters_df.insert(21, "Rank6 condensed", None)


In [11]:
D2_candidates_R1 = D2_candidates[:-3]
D2_candidates_R1

['Sameer Kanal',
 'Dan Ryan',
 'Elana Pirtle-Guiney',
 'Tiffani Penson',
 'Michelle DePass',
 'Nat West',
 'Marnie Glickman',
 'Jonathan Tasini',
 'Bob Simril',
 'Mariah Hudson',
 'Michael (Mike) Marshall',
 'James Armstrong',
 'Chris Olson',
 'Debbie Kitchin',
 'Jennifer Park',
 'Nabil Zaghloul',
 'Will Mespelt',
 'Laura Streib',
 'Reuben Berlin',
 'Liz Taylor',
 'Sam Sachs',
 'Antonio Jamal PettyJohnBlue',
 'Uncertified Write In']

In [12]:
condensed_ballots_R1 = []

for i in tqdm(D2_voters_df.index):
    this_ballot_raw = D2_voters_df.loc[i, ['Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw', 'Rank5 raw', 'Rank6 raw']]
    
    this_ballot_cleaned = [name for name in this_ballot_raw if name in D2_candidates_R1]

    this_ballot_condensed = []
    for name in this_ballot_cleaned:
        if name not in this_ballot_condensed:
            this_ballot_condensed.append(name)    

    if len(this_ballot_condensed) > 0:
        condensed_ballots_R1.append(this_ballot_condensed)
    
    if len(this_ballot_condensed) < 6:
        fill_len = 6 - len(this_ballot_condensed)
        this_ballot_condensed = this_ballot_condensed + ["skipped"]*fill_len

    D2_voters_df.loc[i, "Rank1 condensed"] = this_ballot_condensed[0]
    D2_voters_df.loc[i, "Rank2 condensed"] = this_ballot_condensed[1]
    D2_voters_df.loc[i, "Rank3 condensed"] = this_ballot_condensed[2]
    D2_voters_df.loc[i, "Rank4 condensed"] = this_ballot_condensed[3]
    D2_voters_df.loc[i, "Rank5 condensed"] = this_ballot_condensed[4]
    D2_voters_df.loc[i, "Rank6 condensed"] = this_ballot_condensed[5]

100%|███████████████████████████████████| 77686/77686 [00:31<00:00, 2442.14it/s]


### Create two datasets to save: one with the raw data and one with the condensed data.  Keep all the intro columns for the raw data, but not for the cleaned data.

In [13]:
D2_voters_df.columns[0:22]

Index(['RowNumber', 'BoxID', 'BoxPosition', 'BallotID', 'PrecinctID',
       'BallotStyleID', 'PrecinctStyleName', 'ScanComputerName', 'Status',
       'Remade', 'Rank1 raw', 'Rank2 raw', 'Rank3 raw', 'Rank4 raw',
       'Rank5 raw', 'Rank6 raw', 'Rank1 condensed', 'Rank2 condensed',
       'Rank3 condensed', 'Rank4 condensed', 'Rank5 condensed',
       'Rank6 condensed'],
      dtype='object')

In [14]:
D2_raw_df = D2_voters_df[D2_voters_df.columns[0:16]]

In [15]:
D2_condensed_df = D2_voters_df[['PrecinctStyleName'] + list(D2_voters_df.columns)[16:22]]

### The raw file is good to go; just rename the ranking columns.

In [16]:
D2_raw_df = D2_raw_df.rename(columns = {
    'Rank1 raw' : 'rank1',
    'Rank2 raw' : 'rank2',
    'Rank3 raw' : 'rank3',
    'Rank4 raw' : 'rank4',
    'Rank5 raw' : 'rank5',
    'Rank6 raw' : 'rank6'
})

In [17]:
D2_raw_df.to_csv("./Cleaned CVRs/Portland_D2_Cleaned.csv", index = False)

### For the condensed file, remove voters who no longer have any votes. (These are the ones who only voted for 'Write-in-123', 'Write-in-124', and/or 'Write-in-125'.)  

In [18]:
D2_condensed_df = D2_condensed_df[D2_condensed_df['Rank1 condensed'] != 'skipped'].reset_index(drop="True")

### Rename the columns:

In [19]:
D2_condensed_df = D2_condensed_df.rename(columns = {
    'PrecinctStyleName': 'Precinct',
    'Rank1 condensed' : 'rank1',
    'Rank2 condensed' : 'rank2',
    'Rank3 condensed' : 'rank3',
    'Rank4 condensed' : 'rank4',
    'Rank5 condensed' : 'rank5',
    'Rank6 condensed' : 'rank6'
})

### Change the format of the entries in the Precinct column to match the precinct shapefile:

In [20]:
set(D2_condensed_df['Precinct'])

{'3308-1',
 '4301-1',
 '4302-1',
 '4303-1',
 '4304-1',
 '4305-1',
 '4306-1',
 '4307-1',
 '4401-1',
 '4402-1',
 '4403-1',
 '4404-1',
 '4405-1',
 '4406-1',
 '4501-1',
 '4502-2',
 '4503-2'}

In [21]:
for i in D2_condensed_df.index:
    if D2_condensed_df.loc[i, 'Precinct'].startswith('CLACK'):
        D2_condensed_df.loc[i, 'Precinct'] = 'C'+ D2_condensed_df.loc[i, 'Precinct'][-5:-2]
    elif D2_condensed_df.loc[i, 'Precinct'].startswith('WASH'):
        D2_condensed_df.loc[i, 'Precinct'] = 'W'+ D2_condensed_df.loc[i, 'Precinct'][-5:-2]
    else:
        D2_condensed_df.loc[i, 'Precinct'] = 'M'+ D2_condensed_df.loc[i, 'Precinct'][0:4]

In [22]:
D2_condensed_df

,Precinct,rank1,rank2,rank3,rank4,rank5,rank6
0,M4305,Marnie Glickman,Elana Pirtle-Guiney,Jonathan Tasini,Dan Ryan,skipped,skipped
1,M4305,Marnie Glickman,Dan Ryan,Nabil Zaghloul,skipped,skipped,skipped
2,M3308,Sameer Kanal,Michelle DePass,Tiffani Penson,Nat West,skipped,skipped
3,M4501,Dan Ryan,Tiffani Penson,Nat West,Mariah Hudson,Jonathan Tasini,Elana Pirtle-Guiney
4,M4304,Jennifer Park,Liz Taylor,Bob Simril,Laura Streib,Michelle DePass,Debbie Kitchin
...,...,...,...,...,...,...,...
77152,M4403,Sam Sachs,Will Mespelt,Laura Streib,James Armstrong,Dan Ryan,Nat West
77153,M4302,Jonathan Tasini,Sam Sachs,Will Mespelt,Jennifer Park,Dan Ryan,Laura Streib
77154,M4405,Chris Olson,Michael (Mike) Marshall,Reuben Berlin,Jonathan Tasini,Marnie Glickman,Jennifer Park
77155,M4302,Chris Olson,Jennifer Park,Liz Taylor,skipped,skipped,skipped


### Save it:

In [23]:
D2_condensed_df.to_csv("./Cleaned CVRs/Portland_D2_Condensed.csv", index = False)